In [1]:
import pathlib
import random
import string
import re
import numpy as np
import json
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import tensorflow as tf
import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

In [2]:
text_file = keras.utils.get_file(
     fname="spa-eng.zip",
     origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
     extract=True,
 )
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

In [3]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    eng = "[start] " + eng + " [end]"
    text_pairs.append((spa,eng))

In [4]:
random.shuffle(text_pairs)

In [5]:
text_pairs

[('Haz una lista.', '[start] Make a list. [end]'),
 ('Mary es una linda mujer.', '[start] Mary is a beautiful woman. [end]'),
 ('Puedo hacer cualquier cosa.', '[start] I can do anything. [end]'),
 ('Lo conseguí gratis.', '[start] I got it for free. [end]'),
 ('Él se convirtió en un cantante famoso.',
  '[start] He became a famous singer. [end]'),
 ('Me voy a meter en problemas si hago eso.',
  "[start] I'll get in trouble if I do that. [end]"),
 ('¿Qué harás cuando seas mayor?',
  '[start] What will you do when you grow up? [end]'),
 ('Entró corriendo en la clase.', '[start] He ran into the classroom. [end]'),
 ('También es una preciosidad.', "[start] She's also a beauty. [end]"),
 ('No deberías leer con tan poca luz.',
  "[start] You shouldn't read in such poor light. [end]"),
 ('Ella envió a los niños al colegio.',
  '[start] She sent her children off to school. [end]'),
 ('A Tomás le encanta cocinar.', '[start] Tom loves to cook. [end]'),
 ('Lo que me sorprendió fue que solo diez pe

In [6]:
for _ in range(5):
    print(random.choice(text_pairs))

('Mi padre nunca ha estado en el extranjero.', '[start] My father has never been abroad. [end]')
('No compré nada.', "[start] I didn't buy anything. [end]")
('No seas tan reservado.', "[start] Don't be so reserved. [end]")
('Con tus habilidades, estoy seguro de que no te sería muy difícil encontrar otro trabajo.', "[start] With your skills, I'm sure it wouldn't be too hard to find another job. [end]")
('Todos estaban buscando a Tom.', '[start] Everybody was looking for Tom. [end]')


In [7]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [8]:
# parameters
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# vectorization
spa_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)

train_spa_texts = [pair[0] for pair in train_pairs]
train_eng_texts = [pair[1] for pair in train_pairs]

spa_vectorization.adapt(train_spa_texts)
eng_vectorization.adapt(train_eng_texts)

#save the vectorization layers
spa_vectorization_config = spa_vectorization.get_config()
spa_vectorization_config.pop('standardize', None)
spa_vocab =spa_vectorization.get_vocabulary()
with open('spa_vectorization_config1.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vectorization_config, f)
    
with open('spa_vocab1.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vocab, f)
    
eng_vectorization_config = eng_vectorization.get_config()
eng_vectorization_config.pop('standardize', None)
eng_vocab = eng_vectorization.get_vocabulary()
with open('eng_vectorization_config1.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vectorization_config, f)
    
with open('eng_vocab1.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vocab, f)
    

def format_dataset(spa, eng):
    spa = spa_vectorization(spa)
    eng = eng_vectorization(eng)
    return (
        {
            "encoder_inputs": spa,
            "decoder_inputs": eng[:, :-1],
        },
        eng[:, 1:],
    )
    
def make_dataset(pairs):
    spa_texts, eng_texts = zip(*pairs)
    spa_texts = list(spa_texts)
    eng_texts = list(eng_texts)
    dataset = tf_data.Dataset.from_tensor_slices((spa_texts, eng_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [9]:
def format_dataset(spa, eng):
    spa = spa_vectorization(spa)
    eng = eng_vectorization(eng)
    return (
        {
            "encoder_inputs": spa,
            "decoder_inputs": eng[:, :-1],
        },
        eng[:, 1:],
    )


def make_dataset(pairs):
    spa_texts, eng_texts = zip(*pairs)
    spa_texts = list(spa_texts)
    eng_texts = list(eng_texts)
    dataset = tf_data.Dataset.from_tensor_slices((spa_texts, eng_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [10]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [11]:
text_pairs

[('Ningún hombre podría hacerlo.', '[start] No one man could do it. [end]'),
 ('Tengo harto tiempo para hacer eso.',
  '[start] I have plenty of time to do that. [end]'),
 ('Tengo diez bolígrafos.', '[start] I have ten pens. [end]'),
 ('Te hice una pregunta.', '[start] I asked you a question. [end]'),
 ('Me gusta la atmósfera de este restaurante.',
  '[start] I like the atmosphere of this restaurant. [end]'),
 ('Quiero comer pizza esta noche.',
  '[start] I want to eat pizza tonight. [end]'),
 ('¿Cómo irás a la escuela mañana?',
  '[start] How will you get to school tomorrow? [end]'),
 ('Él envió su solicitud a la oficina.',
  '[start] He sent in his application to the office. [end]'),
 ('Tom no vino al colegio ayer.',
  "[start] Tom didn't come to school yesterday. [end]"),
 ('Beethoven fue un gran compositor.',
  '[start] Beethoven was a great composer. [end]'),
 ('Espero que podamos vernos en algún momento otra vez.',
  "[start] I hope we'll see each other again sometime. [end]"),
 

In [12]:
# Creating an Encoder
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
        else:
            padding_mask = None
            
        attention_output = self.attention(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask,
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config
    
# Creating a Positional Embedding
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0, limit = length, delta = 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        if mask is not None:
            return tf.not_equal(inputs, 0)
        else:
            return None
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "sequence_length": self.sequence_length,
            "embed_dim": self.embed_dim,
        })
        return config
    
# Creating a Decoder
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask, casual_mask)
        else:
            padding_mask = None
            
        attention_output_1 = self.attention_1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )
        
        out_2 = self.layernorm_2(out_1 + attention_output_2)
        proj_output = self.dense_proj(out_2)
        
        return self.layernorm_3(out_2 + proj_output)
    
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, tf.int32)
        mask = tf.reshape(mask,(1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.convert_to_tensor([1, 1]),
            ],
            axis = 0,
        )
        return tf.tile(mask, mult)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config
  
 
      

In [13]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [14]:
epochs = 20 

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  3,845,120 │ encoder_inputs[0… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encoder │ (None, None, 256) │  3,155,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, None,      │ 12,959,640 │ decoder_inputs[0… │
│ (Functional)        │ 15000)            │            │ transformer_enco… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 927s 707ms/step - accuracy: 0.6998 - loss: 2.2353 - val_accuracy: 0.7346 - val_loss: 1.7061
Epoch 2/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 4661s 4s/step - accuracy: 0.7733 - loss: 1.4389 - val_accuracy: 0.9418 - val_loss: 0.4604
Epoch 3/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 3595s 3s/step - accuracy: 0.9390 - loss: 0.4573 - val_accuracy: 0.9822 - val_loss: 0.1612
Epoch 4/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 2161s 2s/step - accuracy: 0.9816 - loss: 0.1520 - val_accuracy: 0.9902 - val_loss: 0.0976
Epoch 5/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 936s 719ms/step - accuracy: 0.9892 - loss: 0.0930 - val_accuracy: 0.9930 - val_loss: 0.0726
Epoch 6/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 2505s 2s/step - accuracy: 0.9934 - loss: 0.0571 - val_accuracy: 0.9947 - val_loss: 0.0578
Epoch 7/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 2711s 2s/step - accuracy: 0.9955 - loss: 0.0394 - val_accuracy: 0.9955 - val_loss: 0.0475
Epoch 8/20
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 2506s 2s/step - accuracy: 0.99

In [15]:
eng_vocab = eng_vectorization.get_vocabulary()
eng_index_lookup = dict(zip(range(len(eng_vocab)), eng_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = spa_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = eng_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = eng_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)
    print("Spanish sentence ",input_sentence)
    print("Predicted english sentence ",translated)

Spanish sentence  Si ves su casa, sabrás que él es pobre.
Predicted english sentence  [start] owe do there quantities [end]
Spanish sentence  ¿Os parece una estupidez?
Predicted english sentence  [start] any it specialization                 
Spanish sentence  Tom es un homicida.
Predicted english sentence  [start] is he fir                 
Spanish sentence  Lo contactaremos, tan pronto descubramos algo.
Predicted english sentence  [start] tallinn about first pretend were               
Spanish sentence  Este libro habla sobre un rey que pierde su corona.
Predicted english sentence  [start] some bad room he usual [end]
Spanish sentence  La Madre Teresa empleó esa ayuda monetaria para ayudar a la gente pobre de la India y de todo el mundo.
Predicted english sentence  [start] still roman operator much understand chapel what plan the you night lend night you definitely his roman very tom tom
Spanish sentence  Estoy deseando conocerle.
Predicted english sentence  [start] cab devil        

In [16]:
transformer.save("spa_to_eng_model.keras")